# Assignment 5 - Analysis and Some Stats

**Course:** Programming for Data Analytics  
**Lecture:** Module 5 : Analysis and Some Stats  
**Author:** Clyde Watts
**Date:** 2025-10-25
**Assignment:** Ireland Population Status


## Assigment Instructions

Upload the notebook assignment05-population.ipynb to you assignments repository.

### Part 1 70%
Write a jupyter notebook that analyses the differences between the sexes by age in Ireland.

Weighted mean age (by sex)
The difference between the sexes by age
This part does not need to look at the regions.

ie You can take the notebook I used in the lectures and substitute the sexes for the regions.

### Part 2 20%
In the same notebook, make a variable that stores an age (say 35).

Write that code that would group the people within 5 years of that age together, into one age group 

Calculate the population difference between the sexes in that age group.

### Part 3 10%
In the same notebook.

Write the code that would work out which region in Ireland has the biggest population difference between the sexes in that age group

Last modified: Thursday, 23 October 2025, 3:30 PM



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## The Differences between the male and female populate by age in Ireland


### Part 1 70%
Write a jupyter notebook that analyses the differences between the sexes by age in Ireland.

Weighted mean age (by sex)
The difference between the sexes by age
This part does not need to look at the regions.

ie You can take the notebook I used in the lectures and substitute the sexes for the regions.

**References**


https://data.cso.ie/#
https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en




**Download the data using URL into Dataframe**


Note: 

This system ( my laptop ) has vscode extension Data Wrangler . This will allow me to explore the data quickly.




In [ ]:
# Download data from URL into Dataframe
URL = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df_raw = pd.read_csv(URL)
# Get column list
print("Data Frame Columns:",df_raw.columns.to_list())
# Get index 
print("Data Frame Index:",df_raw.index)
df_raw.head()

I am interested in the following Columns , Sex , 'Single Year of Age',  'Administrative Counties'

In [ ]:
# Get the sum of values by Sex 
# This will tell me if that Both sexes is a agregation of Female and Male
df_raw.groupby(['Sex']).sum()['VALUE']
# I assume that "Both sexes" is the some of Male and Female 

In [ ]:
# Get the sum of the 'Single Year of Age'
df_raw.groupby(['Single Year of Age'],sort=True).sum()['VALUE']


In [ ]:
# Now get sum of 'Administrative Counties'
df_raw.groupby(['Administrative Counties'],sort=True).sum()['VALUE']

Remove columns that are not required

In [ ]:
drop_list = ['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'CensusYear', 'C02199V02655',  'C02076V03371', 'C03789V04537',
               'UNIT']
df_raw.drop(columns=drop_list,inplace=True)
print("Data Frame Columns after drop:",df_raw.columns.to_list())
df_raw.head()

Now remove all counties except Ireland - the whole country and exclude Both Sexes from Sex ,and All ages from Single Year Of Age

In [ ]:
# now create a data frame for analysis
# Counts == Ireland and Sex != Both and Single Year of Age != All ages
# Note the filtered data frame needed to be copied to avoid SettingWithCopyWarning 
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
# Looking at it from a database perspective we are creating a new table from a select statement , rather than a view
df_population = df_raw[(df_raw['Administrative Counties']=='Ireland') & (df_raw['Sex'] != 'Both sexes') & (df_raw['Single Year of Age'] != 'All ages')].copy()
# Also drop "Ireland" column as it is not needed now
df_population.drop(columns=['Administrative Counties'],inplace=True)
# Rename the VALUE column to Population - Prettier for analysis
df_population.rename(columns={'VALUE':'Population'},inplace=True)
df_population.head()

Add another column called Year No - which is the year number . 
- Under 1 Year will be Zero

In [ ]:
# First replace the 'Under 1 year' with '0' as per lecture instructions
df_population["Year No"] = df_population['Single Year of Age'].replace({'Under 1 year': '0'})
# Now extract the digit from the year number string and convert to integer
df_population['Year No'] = df_population['Year No'].str.extract('(\d+)',expand=False).astype(int)
df_population.head()


### Part 1.1  Calculate The "Weighted" Mean Per Sex

There are a couple of approaches

In [ ]:
# Do the weighted for male first
male_weighted_mean = np.average(df_population[df_population['Sex'] == 'Male']['Year No'], 
                                weights=df_population[df_population['Sex'] == 'Male']['Population'])
# Do the weighted for female next
female_weighted_mean = np.average(df_population[df_population['Sex'] == 'Female']['Year No'], 
                                  weights=df_population[df_population['Sex'] == 'Female']['Population'])
print("Weighted Mean Age by Sex:")
print(f"Male: {male_weighted_mean}")
print(f"Female: {female_weighted_mean}")

In [ ]:
# A more manual method , the np method is counter intuitive
# explicitly work out the weighted age
df_population['weighted_age'] = df_population['Year No'] * df_population['Population']
# Now do the group by sum and divide
weighted_mean_by_sex = df_population.groupby('Sex').sum()['weighted_age'] / df_population.groupby('Sex').sum()['Population']
print("Weighted Mean Age by Sex:")
print(weighted_mean_by_sex)


## Part 1.2 The difference between the sexes by age

The assumtion is we subtract the one sex Female from the other Male

In [ ]:
# Pivot data on sex so that we have year no , and population for both male and female
df_population_sex_pivot = df_population.pivot(index='Year No', columns='Sex', values='Population').reset_index()
# create a extra column for difference
df_population_sex_pivot['Difference'] = df_population_sex_pivot['Male'] - df_population_sex_pivot['Female']
df_population_sex_pivot

__Plot a graph with the difference__

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
# Plot the populations
sns.lineplot(data=df_population_sex_pivot, x='Year No', y='Male', ax=ax[0], label='Male', color='blue')
sns.lineplot(data=df_population_sex_pivot, x='Year No', y='Female', ax=ax[0], label='Female', color='pink')
ax[0].set_title('Population by Age and Sex')

# Plot the difference
sns.lineplot(data=df_population_sex_pivot, x='Year No', y='Difference', ax=ax[1], color='green')
# plot the positive numbers blue negative pink
ax[1].fill_between(df_population_sex_pivot['Year No'], df_population_sex_pivot['Difference'], where=(df_population_sex_pivot['Difference'] > 0), color='blue', alpha=0.5)
ax[1].fill_between(df_population_sex_pivot['Year No'], df_population_sex_pivot['Difference'], where=(df_population_sex_pivot['Difference'] < 0), color='pink', alpha=0.5)
ax[1].set_title('Population Difference (Male - Female) by Age')
plt.tight_layout()
plt.show()


## Part 2: Ten Year Window
### Part 2 20%
In the same notebook, make a variable that stores an age (say 35).

Write that code that would group the people within 5 years of that age together, into one age group 

Calculate the population difference between the sexes in that age group.

In the example , for an age 35 the range starts from 31 ( 35-4 ) and ( 35 + 4) 39 . The 35 year is included in both directions , thus it spans 9 years.  The window will be reduces if there are no ages below or above the range , example 100 will be 100-4 to 100. , and 0 will be 0 to 0 + 4.

Using the following function to do the calculation , this is equivalent to a ordered analytic function in SQL
The following has a window - the range , and if the range is centered . As well as min_periods which allows for partial windows.

An alternative is to use a cyclic window , that is use a window which wraps around from 0 to 100 in a cyclic manner 

In [ ]:
# Some ytest metrics to validate the rolling window
df_population_sex_pivot['One']=1
df_population_sex_pivot['Increasing Age Group'] = df_population_sex_pivot.index + 1
df_population_sex_pivot_following = df_population_sex_pivot[['Male', 'Female','One','Increasing Age Group']].rolling(window=9, center=True, min_periods=0).sum()
df_population_sex_pivot_following['Difference'] = df_population_sex_pivot_following['Male'] - df_population_sex_pivot_following['Female']
df_population_sex_pivot_following

To get the N year , the Nth record is retrieved from the data frame

In [ ]:
# The Nth year record
Nth_year = 35
# Calculate the population difference in the 9 year window centered on Nth_year
print(f"Population difference in 9 year window centered on age {Nth_year}:")
print(f"                                             Male: {df_population_sex_pivot_following.loc[Nth_year, 'Male']}")
print(f"                                             Female: {df_population_sex_pivot_following.loc[Nth_year, 'Female']}")
print(f"                                             Difference: {df_population_sex_pivot_following.loc[Nth_year, 'Difference']}")  

### Country / Agg Difference - Part 3 10%
In the same notebook.

Write the code that would work out which region in Ireland has the biggest population difference between the sexes in that age group

Last modified: Thursday, 23 October 2025, 3:30 PM

https://medium.com/@heyamit10/pandas-dense-rank-a-simple-guide-for-beginners-bf9889728b66


### Part 3.1 - Difference calculation

In [ ]:
df_raw

In [ ]:
# create a new data frame excluding the Sex Both sexes , and Single Year of Age != All ages and ireland
# Use query syntax to cleanup - picked this up in Data Camp , and I am familiar with it from SQL
df_country_agg_difference = df_raw.query("not(`Administrative Counties` == 'Ireland' or `Sex` == 'Both sexes' or `Single Year of Age` == 'All ages')").copy()
# Rename the VALUE column to Population - Prettier for analysis
df_country_agg_difference.rename(columns={'VALUE':'Population'},inplace=True)
# Pivot the data , then drop index , don't want to use multi index
df_country_agg_difference = df_country_agg_difference.pivot(index=['Administrative Counties','Single Year of Age'], columns=['Sex'], values='Population')
# Next work out the difference , in this case take the absolute difference
df_country_agg_difference['Difference'] = abs(df_country_agg_difference['Male'] - df_country_agg_difference['Female'])
# Splits the multi index back to columns
df_country_agg_difference.reset_index(inplace=True)
# Sort by differences descending , top country and age groups with biggest difference
df_country_agg_difference.sort_values(by='Difference', ascending=False,inplace=True)
df_country_agg_difference

A better alternative is to use dense rank . If there are two candidates for counties with the same biggest difference. This should resolve this scenario

In [ ]:
# Rank will give a number based on the increasing order of the difference
# If there is a tie then both will get the same range and the next number of ties will be skipped
# dense rank does not skip numbers in the ranking , may not make any difference in this case because we are only looking for the top one
df_country_agg_difference["dense_rank"] = df_country_agg_difference["Difference"].rank(method='dense', ascending=False)
print("Countries and Age groups with the biggest population difference between the sexes:")
print(f" {df_country_agg_difference[df_country_agg_difference['dense_rank'] == 1]}")
df_country_agg_difference.sort_values(by='Difference', ascending=False,inplace=True)
df_country_agg_difference

__The End__